# Imports

In [2]:
# 导入需要的库    Импорт требуемых библиотек
import keras
import numpy as np
import pandas as pd
import transformers
import tensorflow as tf
import tqdm.notebook as tqdm
import matplotlib.pyplot as plt
from transformers import GPT2Tokenizer
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
from transformers import DataCollatorWithPadding
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
# 调用 GPU 加速    Вызов GPU-ускорения
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except:
        pass

# Dataset

In [4]:
# 文件路径    Путь к файлу
file_path = r"C:\Users\lcf14\Desktop\homework\Machine_Learning_appli\有俄语-英语对应句 - 2024-10-31.tsv"

In [1]:
# 读取文件并设置列名，忽略有问题的行    Чтение файла и установка имен столбцов, пропуская строки с ошибками
data = pd.read_csv(file_path, sep="\t", header=None, names=["id_rus", "text_rus", "id_eng", "text_eng"], on_bad_lines='skip')
data = data[:5000]  # 仅选择前10000行数据

# 输出前几行查看数据    Вывод первых нескольких строк для просмотра данных
print(data[['id_rus', 'text_rus', 'id_eng', 'text_eng']].head())

NameError: name 'pd' is not defined

In [6]:
"""class Tokenizer(transformers.GPT2Tokenizer):

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        if token_ids_1 is None:
            return [self.bos_token_id, *token_ids_0, self.eos_token_id]

        return [self.bos_token_id, *token_ids_0, self.bos_token_id, *token_ids_1, self.eos_token_id]"""

'class Tokenizer(transformers.GPT2Tokenizer):\n\n    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):\n        if token_ids_1 is None:\n            return [self.bos_token_id, *token_ids_0, self.eos_token_id]\n\n        return [self.bos_token_id, *token_ids_0, self.bos_token_id, *token_ids_1, self.eos_token_id]'

In [7]:
# 初始化一个GPT2的分词器    Инициализация токенизатора GPT-2
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# 添加特殊标记，包括起始、结束和填充标记    Добавление специальных токенов: начало, конец и заполнение
tokenizer.add_special_tokens({
    "bos_token": "<BOS>",  # 起始标记    Токен начала предложения
    "eos_token": "<EOS>",  # 结束标记    Токен конца предложения
    "pad_token": "<PAD>"   # 填充标记    Токен заполнения
})

# 将填充标记的ID设为结束标记的ID    Установка идентификатора токена заполнения таким же, как у токена конца предложения
tokenizer.pad_token_id = tokenizer.eos_token_id

In [8]:
# 定义数据集生成函数    Определение функции для генерации датасета
def Sentences_Dataset(sentence_pairs, tokenizer, max_len=24):
    inputs = []  # 输入张量列表    Список тензоров для входов
    outputs = []  # 输出张量列表    Список тензоров для выходов
    predicted_outputs = []  # 预测输出张量列表    Список тензоров для предсказанных выходов
    masks = []  # 掩码张量列表    Список тензоров для масок

    # 使用 values 属性快速访问数据    Использование свойства values для быстрого доступа к данным
    for i, row in enumerate(sentence_pairs.values):
        # 对输入（俄语）和输出（英语）进行分词    Токенизация входных данных (русский) и выходных данных (английский)
        input_tokens = tokenizer(row[1], max_length=max_len, padding='max_length', truncation=True, add_special_tokens=True)['input_ids']
        output_tokens = tokenizer(row[3], max_length=max_len - 1, padding='max_length', truncation=True, add_special_tokens=True)['input_ids']

        # 在输出序列前加上 BOS token    Добавление BOS-токена в начало выходной последовательности
        output_tokens_with_bos = [tokenizer.bos_token_id] + output_tokens
        # 在预测输出序列末尾加上 EOS token    Добавление EOS-токена в конец предсказанной выходной последовательности
        predicted_output_tokens = output_tokens + [tokenizer.eos_token_id]

        # 将 tokens 添加到列表中    Добавление токенов в списки
        inputs.append(torch.tensor(input_tokens))
        outputs.append(torch.tensor(output_tokens_with_bos))
        predicted_outputs.append(torch.tensor(predicted_output_tokens))  # 预测输出是没有 BOS token 的版本    Предсказанная выходная последовательность не содержит BOS-токена

        # 生成 mask，标记 pad 的位置为 0    Генерация маски, помечая места заполнения (pad) нулями
        masks.append(torch.tensor([1 if token != tokenizer.pad_token_id else 0 for token in output_tokens_with_bos]))

    # 使用 torch.stack 将列表转换为 tensor    Преобразование списков в тензоры с помощью torch.stack
    inputs = torch.stack(inputs)
    outputs = torch.stack(outputs)
    predicted_outputs = torch.stack(predicted_outputs)
    masks = torch.stack(masks)

    return (inputs, outputs), predicted_outputs, masks


# 使用自定义数据集函数生成数据    Генерация данных с помощью функции пользовательского датасета
(inputs, outputs), predicted_output, masks = Sentences_Dataset(data[['id_rus', 'text_rus', 'id_eng', 'text_eng']], tokenizer)
print("Input tensor shape:", inputs.shape)       # 输入张量的形状    Форма входного тензора
print("Output tensor shape:", outputs.shape)     # 输出张量的形状    Форма выходного тензора
print("Mask tensor shape:", masks.shape)         # 掩码张量的形状    Форма тензора маски
print("Predicted Output tensor shape:", predicted_output.shape)  # 预测输出张量的形状    Форма тензора предсказанного выхода

Input tensor shape: torch.Size([735543, 32])
Output tensor shape: torch.Size([735543, 32])
Mask tensor shape: torch.Size([735543, 32])
Predicted Output tensor shape: torch.Size([735543, 32])


In [9]:
# 定义自定义数据集类    Определение класса пользовательского датасета
class TranslationDataset(Dataset):
    def __init__(self, sentence_pairs, tokenizer, max_len=24):
        self.sentence_pairs = sentence_pairs  # 存储句子对数据    Хранение пар предложений
        self.tokenizer = tokenizer  # 存储分词器    Хранение токенизатора
        self.max_len = max_len  # 最大序列长度    Максимальная длина последовательности

        # 调用外部的 create_dataset 函数生成数据    Вызов внешней функции create_dataset для генерации данных
        (self.inputs, self.outputs), self.predicted_outputs, self.masks = Sentences_Dataset(sentence_pairs, tokenizer, max_len)

    def __len__(self):
        return len(self.inputs)  # 返回数据集的大小    Возвращение размера датасета

    def __getitem__(self, idx):
        return {  # 返回指定索引的数据    Возвращение данных по указанному индексу
            'input_ids': self.inputs[idx],  # 输入 ID    Входные ID
            'output_ids': self.outputs[idx],  # 输出 ID    Выходные ID
            'predicted_output_ids': self.predicted_outputs[idx],  # 预测输出 ID    Предсказанные выходные ID
            'attention_mask': self.masks[idx]  # 注意力掩码    Маска внимания
        }


# 创建数据集对象    Создание объекта датасета
translation_dataset = TranslationDataset(data[['id_rus', 'text_rus', 'id_eng', 'text_eng']], tokenizer)

# 创建数据加载器    Создание загрузчика данных
batch_size = 256  # 根据显存大小调整批量大小    Настройка размера батча в зависимости от доступной памяти
data_loader = DataLoader(translation_dataset, batch_size=batch_size, shuffle=True)  # 创建数据加载器对象    Создание объекта загрузчика данных

# 检查数据加载器    Проверка загрузчика данных
for batch in data_loader:
    print("Input IDs shape:", batch['input_ids'].shape)  # 打印输入 ID 的形状    Вывод формы входных ID
    print("Output IDs shape:", batch['output_ids'].shape)  # 打印输出 ID 的形状    Вывод формы выходных ID
    print("Predicted Output IDs shape:", batch['predicted_output_ids'].shape)  # 打印预测输出 ID 的形状    Вывод формы предсказанных выходных ID
    print("Attention mask shape:", batch['attention_mask'].shape)  # 打印注意力掩码的形状    Вывод формы маски внимания
    break  # 只查看第一个批次    Просмотр только первого батча

# 打印第一个样本的输入、输出、预测输出和注意力掩码    
# Вывод входных, выходных, предсказанных выходных данных и маски внимания для первого примера
for batch in data_loader:
    print("input_ids:", batch['input_ids'][0])  # 打印第一个样本的输入 IDs    Вывод входных ID первого примера
    print("output_ids:", batch['output_ids'][0])  # 打印第一个样本的输出 IDs    Вывод выходных ID первого примера
    print("Predicted Output IDs:", batch['predicted_output_ids'][0])  # 打印第一个样本的预测输出 IDs    Вывод предсказанных выходных ID первого примера
    print("attention_mask:", batch['attention_mask'][0])  # 打印第一个样本的注意力掩码    Вывод маски внимания первого примера
    break

Input IDs shape: torch.Size([256, 32])
Output IDs shape: torch.Size([256, 32])
Predicted Output IDs shape: torch.Size([256, 32])
Attention mask shape: torch.Size([256, 32])
input_ids: tensor([  140,    94, 43666, 16843, 30143, 16142,   140,   117,   220, 20375,
        15166,    11, 12466,   122,   220,   141,   229,   141,   239, 43108,
        12466,    95, 25443,   120, 12466,   123, 21169, 15166, 21727, 18849,
        20375,    13])
output_ids: tensor([50257,  5211,   644,  4186,  7893,    13, 50258, 50258, 50258, 50258,
        50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
        50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
        50258, 50258])
Predicted Output IDs: tensor([ 5211,   644,  4186,  7893,    13, 50258, 50258, 50258, 50258, 50258,
        50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
        50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258,
        50258, 50258])
attention_m

# Model

In [22]:
# 定义编码器类    Определение класса кодировщика
class Encoder(nn.Module):
    def __init__(self, units, n_tokens, n_stacks=1, bidirectional=False, dropout_rate=0.3, cell_type=nn.GRU):
        # 初始化编码器    Инициализация кодировщика
        super(Encoder, self).__init__()
        # 定义嵌入层，用于将输入序列转为嵌入向量    Определение слоя внедрения для преобразования входной последовательности в вектор внедрения
        self.embedding = nn.Embedding(n_tokens, units)
        # 定义 RNN 层（GRU），根据是否双向设置隐藏单元的数量    Определение слоя RNN (GRU), с настройкой количества скрытых единиц в зависимости от того, является ли он двунаправленным
        self.rnn = cell_type(units, units, num_layers=n_stacks, bidirectional=bidirectional, dropout=dropout_rate, batch_first=True)
        # 保存 RNN 是否为双向的    Хранение информации о том, является ли RNN двунаправленным
        self.bidirectional = bidirectional
        # 保存隐藏单元的数量    Хранение количества скрытых единиц
        self.units = units
        # 在 Encoder 和 Decoder 的 __init__ 方法中添加嵌入层初始化    Инициализация весов слоя внедрения в методах __init__ Encoder и Decoder
        nn.init.normal_(self.embedding.weight, mean=0, std=0.01)

    def forward(self, input_seq):
        # 获取输入序列，将其转为嵌入向量    Преобразование входной последовательности в вектор внедрения
        embedded = self.embedding(input_seq)  # (batch_size, seq_len, units)    (размер_пакета, длина_последовательности, количество_единиц)
        # 将嵌入向量输入到 RNN 层    Ввод вектора внедрения в слой RNN
        encoder_outputs, hidden = self.rnn(embedded)

        # 如果 RNN 是双向的，将最后一层的前向和后向隐藏状态拼接    
        # Если RNN двунаправленный, объединение скрытых состояний из последнего слоя по направлению вперед и назад
        if self.bidirectional:
            hidden = torch.cat((hidden[-2], hidden[-1]), dim=1).unsqueeze(0)  # 拼接最后一层的前向和后向隐藏状态    Объединение последних скрытых состояний из обоих направлений
        else:
            hidden = hidden[-1].unsqueeze(0)  # 如果是单向，只取最后一层的隐藏状态    Если однонаправленный, взять только скрытое состояние из последнего слоя

        hidden = hidden[:, :, :self.units]  # 确保hidden维度与解码器一致    Обеспечение соответствия размерности hidden с декодером
        # 确保返回的隐藏状态形状符合解码器的输入需求    
        # Обеспечение того, чтобы возвращаемая форма скрытого состояния соответствовала требованиям для входа в декодер
        return encoder_outputs, hidden[:, :, :self.units]  # 将hidden的维度匹配到解码器的输入形状    Приведение размерности hidden к форме, необходимой для декодера


# 定义解码器类    Определение класса декодера
class Decoder(nn.Module):
    def __init__(self, units, n_labels, attention_dim, embedding_dim, cell_type=nn.GRU):
        # 初始化解码器    Инициализация декодера
        super(Decoder, self).__init__()
        # 定义嵌入层，用于将解码器输入转换为嵌入向量    Определение слоя внедрения для преобразования входа в декодер в вектор внедрения
        self.embedding = nn.Embedding(n_labels, embedding_dim)
        # 定义 RNN 层，用于处理嵌入向量和上下文向量    Определение слоя RNN для обработки вектора внедрения и контекстного вектора
        self.rnn = cell_type(embedding_dim, units, batch_first=True)
        # 定义输出层，用于生成解码器的最终预测    Определение выходного слоя для генерации окончательного предсказания декодера
        self.fc_out = nn.Linear(units, n_labels)
        # 初始化嵌入层权重    Инициализация весов слоя внедрения
        nn.init.normal_(self.embedding.weight, mean=0, std=0.01)

    def forward(self, decoder_input, hidden):
        # 获取解码器输入的嵌入向量    Получение вектора внедрения для входа в декодер
        decoder_input_embedded = self.embedding(decoder_input)
        # 如果 hidden 是 None，初始化为零张量    Если hidden равно None, инициализировать его как нулевой тензор
        if hidden is None:
            hidden = torch.zeros(1, decoder_input.size(0), self.rnn.hidden_size, device=decoder_input.device)
        # 确保隐藏状态是连续的    Обеспечение того, чтобы скрытое состояние было непрерывным
        hidden = hidden.contiguous()
        # 将嵌入向量输入到 RNN 层    Ввод вектора внедрения в слой RNN
        output, hidden = self.rnn(decoder_input_embedded, hidden)
        # 将 RNN 的输出传入全连接层生成预测    Пропуск выходных данных RNN через полносвязный слой для получения предсказания
        prediction = self.fc_out(output.squeeze(1))
        # 返回预测结果和新的隐藏状态    Возвращение предсказания и нового скрытого состояния
        return prediction, hidden


# 定义序列到序列模型类    Определение класса Seq2Seq модели
class Seq2SeqModel(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2SeqModel, self).__init__()
        self.encoder = encoder  # 设置编码器    Установка кодировщика
        self.decoder = decoder  # 设置解码器    Установка декодера

    def forward(self, encoder_input, decoder_input, teacher_forcing_ratio=0.5):
        encoder_outputs, hidden = self.encoder(encoder_input)  # 编码输入序列    Кодирование входной последовательности
         # 初始化输出序列    Инициализация выходной последовательности
        outputs = torch.zeros(decoder_input.size(0), decoder_input.size(1), self.decoder.fc_out.out_features).to(decoder_input.device)

        # 设置解码器的第一个输入标记为 <BOS>    Установка первого входного маркера для декодера как <BOS>
        input_token = decoder_input[:, 0].unsqueeze(1)

        # 逐时间步遍历目标序列长度    Проход по целевой последовательности по временным шагам
        for t in range(1, decoder_input.size(1)):
            # 解码器前向传播    Прямой проход декодера
            output, hidden = self.decoder(input_token, hidden)
            outputs[:, t, :] = output  # 存储当前时间步的输出    Сохранение выходных данных на текущем временном шаге

            # 使用教师强制决定下一个输入    Применение принудительного обучения для выбора следующего входа
            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            top1 = output.argmax(1).unsqueeze(1)  # 获取预测标记    Получение предсказанного маркера
            input_token = decoder_input[:, t].unsqueeze(1) if teacher_force else top1

        return outputs  # 返回解码器的所有输出    Возвращение всех выходных данных декодера

In [23]:
# 定义模型的参数    Определение параметров модели
units = 256  # 隐藏单元的数量    Количество скрытых единиц
n_tokens = len(tokenizer.get_vocab())  # 输入词典大小    Размер словаря для входных данных
n_labels = len(tokenizer.get_vocab())  # 输出词典大小    Размер словаря для выходных данных
n_stacks = 3  # RNN 层数    Количество слоев RNN

# 初始化编码器，传入词典大小、隐藏单元数量、层数等参数    
# Инициализация кодировщика с передачей размера словаря, количества скрытых единиц, количества слоев и других параметров
encoder = Encoder(
    units=units,  # 隐藏单元数量    Количество скрытых единиц
    n_tokens=n_tokens,  # 输入词典大小    Размер словаря для входных данных
    n_stacks=n_stacks,  # RNN 层数    Количество слоев RNN
    bidirectional=True,  # 是否双向    Является ли кодировщик двунаправленным
    cell_type=nn.GRU  # RNN 类型    Тип RNN
)

# 初始化解码器，传入隐藏单元数量、输出词典大小、层数等参数    
# Инициализация декодера с передачей количества скрытых единиц, размера выходного словаря и других параметров
decoder = Decoder(
    units=units,  # 隐藏单元数量    Количество скрытых единиц
    n_labels=n_labels,  # 输出词典大小    Размер выходного словаря
    attention_dim=units,    # 注意力层的维度    Размерность слоя внимания
    embedding_dim=units,    # 嵌入层的维度    Размерность слоя внедрения
    cell_type=nn.GRU       # RNN 的类型    Тип RNN
)

# 初始化 Seq2SeqModel，将编码器和解码器实例传入    Инициализация модели Seq2Seq, передавая экземпляры кодировщика и декодера
seq2seq_model = Seq2SeqModel(
    encoder=encoder,  # 传入编码器    Передача кодировщика
    decoder=decoder   # 传入解码器    Передача декодера
)

# 打印模型结构    Печать структуры модели
print(seq2seq_model)

Seq2SeqModel(
  (encoder): Encoder(
    (embedding): Embedding(50260, 256)
    (rnn): GRU(256, 256, num_layers=3, batch_first=True, dropout=0.3, bidirectional=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(50260, 256)
    (rnn): GRU(256, 256, batch_first=True)
    (fc_out): Linear(in_features=256, out_features=50260, bias=True)
  )
)


# Training

In [24]:
import gc  # 导入垃圾回收模块    Импорт модуля для управления сбором мусора

# 清除缓存和显存的函数    Функция для очистки кэша и видеопамяти
def clear_cache():
    gc.collect()  # 清理 Python 的垃圾内存    Очистка мусора в Python
    if torch.cuda.is_available():  # 如果 GPU 可用    Если доступен GPU
        torch.cuda.empty_cache()  # 清理 GPU 的缓存    Очистка кэша GPU

In [25]:
# 每次运行训练代码前调用此函数    Вызываем эту функцию перед запуском каждого этапа тренировки
clear_cache()  # 清理缓存    Очистка кэша

# 定义模型的训练函数    Определение функции тренировки модели
def train_model(data_loader, model, criterion, optimizer, num_epochs, teacher_forcing_ratio, decay_factor=0.8):
    model.train()  # 设置模型为训练模式    Перевод модели в режим тренировки

    # 开始迭代训练    Начало итерации тренировки
    for epoch in range(num_epochs):
        total_loss = 0  # 总损失    Общая потеря
        total_correct = 0  # 总正确预测    Общее количество правильных предсказаний
        total_tokens = 0  # 总标记数量    Общее количество токенов

        # 每个 epoch 后将教师强制比率逐渐降低    Уменьшаем коэффициент принудительного учителя после каждого этапа
        current_teacher_forcing_ratio = teacher_forcing_ratio * (decay_factor ** epoch)

        for batch in tqdm(data_loader, desc=f"Epoch [{epoch + 1}/{num_epochs}]"):  # 使用 tqdm 显示进度条
            input_ids = batch['input_ids'].to(device)  # 输入序列    Входные последовательности
            output_ids = batch['output_ids'].to(device)  # 输出序列    Выходные последовательности
            attention_mask = batch['attention_mask'].to(device)  # 注意力掩码    Маска внимания

            optimizer.zero_grad()  # 清除梯度    Очистка градиентов

            # 通过模型进行前向传播    Прямое распространение через модель
            predictions = model(input_ids, output_ids, current_teacher_forcing_ratio)

            # 计算损失    Вычисление потерь
            loss = criterion(predictions.view(-1, predictions.size(-1)), output_ids.view(-1))
            loss.backward()  # 反向传播    Обратное распространение
            #torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)  # 梯度裁剪    Обрезка градиента
            optimizer.step()  # 更新模型参数    Обновление параметров модели

            # 累积损失    Накопление потерь
            total_loss += loss.item()

            # 计算准确率    Вычисление точности
            predicted_tokens = predictions.argmax(-1)  # 获取每个时间步的预测标记    Получение предсказанных токенов на каждом шаге времени
            correct = ((predicted_tokens == output_ids) * attention_mask).float().sum()  # 计算正确预测    Вычисление правильных предсказаний
            total_correct += correct.item()
            total_tokens += attention_mask.sum().item()

        # 清理每个 epoch 后的缓存    Очистка кэша после каждого этапа
        clear_cache()

        # 计算并输出每个 epoch 的损失和准确率    Вычисление и вывод потерь и точности после каждого этапа
        epoch_loss = total_loss / len(data_loader)  # 计算当前 epoch 的平均损失    Вычисление средней потери на этапе
        epoch_accuracy = total_correct / total_tokens  # 计算当前 epoch 的准确率    Вычисление точности на этапе
        print(f"Epoch [{epoch + 1}/{num_epochs}]\n Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")

In [34]:
# 设定训练参数    Установка параметров тренировки
num_epochs = 2  # 训练的 epoch 数    Количество эпох тренировки
learning_rate = 0.001  # 学习率    Коэффициент обучения
teacher_forcing_ratio = 0.8  # 教师强制的比率    Коэффициент принудительного учителя

# 损失函数和优化器    Функция потерь и оптимизатор
criterion = nn.CrossEntropyLoss(ignore_index=50258)  # 使用交叉熵损失函数，忽略 padding 标记    Использование функции потерь кросс-энтропии с игнорированием токенов паддинга
optimizer1 = optim.Adam(seq2seq_model.parameters(), lr=learning_rate)  # 使用 Adam 优化器    Использование оптимизатора Adam

# 将模型移动到 GPU    Перенос модели на GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # 检查是否可以使用 GPU    Проверка, доступен ли GPU
seq2seq_model.to(device)  # 将模型加载到选择的设备    Перенос модели на выбранное устройство (GPU или CPU)

Seq2SeqModel(
  (encoder): Encoder(
    (embedding): Embedding(50260, 256)
    (rnn): GRU(256, 256, num_layers=3, batch_first=True, dropout=0.3, bidirectional=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(50260, 256)
    (rnn): GRU(256, 256, batch_first=True)
    (fc_out): Linear(in_features=256, out_features=50260, bias=True)
  )
)

In [13]:
# 设定训练参数    Установка параметров тренировки
num_epochs = 2  # 训练的 epoch 数    Количество эпох тренировки
learning_rate = 0.001  # 学习率    Коэффициент обучения
teacher_forcing_ratio = 0.8  # 教师强制的比率    Коэффициент принудительного учителя

# 损失函数和优化器    Функция потерь и оптимизатор
criterion = nn.CrossEntropyLoss(ignore_index=50258)  # 使用交叉熵损失函数，忽略 padding 标记    Использование функции потерь кросс-энтропии с игнорированием токенов паддинга
optimizer1 = optim.Adam(seq2seq_model.parameters(), lr=learning_rate)  # 使用 Adam 优化器    Использование оптимизатора Adam

# 将模型移动到 GPU    Перенос модели на GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # 检查是否可以使用 GPU    Проверка, доступен ли GPU
seq2seq_model.to(device)  # 将模型加载到选择的设备    Перенос модели на выбранное устройство (GPU или CPU)

# 开始训练模型    Начало тренировки модели
train_model(data_loader, seq2seq_model, criterion, optimizer1, num_epochs, teacher_forcing_ratio, decay_factor=0.8)  # 调用训练函数    Вызов функции тренировки

Epoch [1/2]: 100%|███████████████████████████████████████████████████████████████| 2874/2874 [6:45:59<00:00,  8.48s/it]


Epoch [1/2]
 Loss: 5.0818, Accuracy: 0.2502


Epoch [2/2]: 100%|███████████████████████████████████████████████████████████████| 2874/2874 [6:24:28<00:00,  8.03s/it]


Epoch [2/2]
 Loss: 4.0391, Accuracy: 0.3974


# Add attention layers

In [26]:
# 定义编码器类    Определение класса кодировщика
class EncoderWithAttention(nn.Module):
    def __init__(self, units, n_tokens, n_stacks=1, bidirectional=False, dropout_rate=0.3, cell_type=nn.GRU, padding_idx=50258):
        # 初始化编码器    Инициализация кодировщика
        super(EncoderWithAttention, self).__init__()
        # 定义嵌入层，用于将输入序列转为嵌入向量    Определение слоя встраивания для преобразования входных последовательностей в векторные представления
        self.embedding = nn.Embedding(n_tokens, units, padding_idx=padding_idx)
        # 定义 RNN 层（GRU），根据是否双向设置隐藏单元的数量    Определение RNN слоя (GRU), количество скрытых единиц зависит от направления (двусторонний или нет)
        self.rnn = cell_type(units, units, num_layers=n_stacks, bidirectional=bidirectional, dropout=dropout_rate, batch_first=True)
        # 保存 RNN 是否为双向的    Сохранение информации о том, является ли RNN двусторонним
        self.bidirectional = bidirectional
        # 保存隐藏单元的数量    Сохранение количества скрытых единиц
        self.units = units
        # 在 Encoder 和 Decoder 的 __init__ 方法中添加嵌入层初始化    Инициализация весов слоя встраивания в методах __init__ кодировщика и декодировщика
        nn.init.normal_(self.embedding.weight, mean=0, std=0.01)

    def forward(self, input_seq):
        # 获取输入序列，将其转为嵌入向量    Преобразование входной последовательности в векторные представления
        embedded = self.embedding(input_seq)  # (batch_size, seq_len, units)
        # 将嵌入向量输入到 RNN 层    Пропуск векторных представлений через RNN слой
        encoder_outputs, hidden = self.rnn(embedded)
        if encoder_outputs.size(-1) != hidden.size(-1):
            encoder_outputs = encoder_outputs[:, :, :hidden.size(-1)]
        
        # 如果 RNN 是双向的，将最后一层的前向和后向隐藏状态拼接    Если RNN двусторонний, соединяем последние скрытые состояния прямого и обратного направлений
        if self.bidirectional:
            hidden = torch.cat((hidden[-2], hidden[-1]), dim=1).unsqueeze(0)
        else:
            hidden = hidden[-1].unsqueeze(0)

        hidden = hidden[:, :, :self.units]  # 确保hidden维度与解码器一致    Обеспечиваем соответствие размеров hidden для декодера
        # 确保返回的隐藏状态形状符合解码器的输入需求    Обеспечиваем, чтобы возвращаемое скрытое состояние соответствовало требованиям декодера
        return encoder_outputs, hidden[:, :, :self.units]  # 返回编码器输出和隐藏状态    Возвращаем выводы кодировщика и скрытое состояние


# 定义注意力机制类    Определение класса механизма внимания
class Attention(nn.Module):
    def __init__(self, units):
        super(Attention, self).__init__()
        self.units = units

    def dot_score(self, hidden_state, encoder_states):
        # 计算注意力分数    Вычисление оценок внимания
        return torch.sum(hidden_state * encoder_states, dim=2)

    def forward(self, hidden, encoder_outputs, mask):
        # 计算注意力权重    Вычисление весов внимания
        attention_scores = self.dot_score(hidden, encoder_outputs)
        attention_scores = attention_scores.t()

        # 使用掩码，避免网络关注 <pad> tokens    Применение маски, чтобы избежать внимания к <pad> токенам
        attention_scores = attention_scores.masked_fill(mask == 0, -1e5)

        # 返回注意力分数的softmax    Возвращаем softmax от оценок внимания
        return F.softmax(attention_scores, dim=1).unsqueeze(1)


# 定义解码器类    Определение класса декодировщика
class DecoderWithAttention(nn.Module):
    def __init__(self, units, n_labels, attention_dim, embedding_dim, cell_type=nn.GRU):
        super(DecoderWithAttention, self).__init__()
        
        # 定义嵌入层    Определение слоя встраивания
        self.embedding = nn.Embedding(n_labels, embedding_dim)
        # 定义RNN层（GRU）    Определение RNN слоя (GRU)
        self.rnn = cell_type(embedding_dim, units, batch_first=True)
        # 定义输出层    Определение слоя для выхода
        self.fc_out = nn.Linear(units * 2, n_labels)
        
        # 初始化嵌入层的权重    Инициализация весов слоя встраивания
        nn.init.normal_(self.embedding.weight, mean=0, std=0.01)
        
        # 初始化注意力机制    Инициализация механизма внимания
        self.attn = Attention(units)  # 初始化注意力机制    Инициализация механизма внимания
        # 定义上下文向量的线性转换层    Определение линейного преобразования контекстного вектора
        self.context_linear = nn.Linear(units, units)

    def forward(self, decoder_input, hidden, encoder_outputs, mask):
        # 获取解码器输入并嵌入    Получаем входные данные для декодера и преобразуем их в векторное представление
        decoder_input_embedded = self.embedding(decoder_input)
        
        hidden = hidden.contiguous()  # 确保隐藏状态是连续的    Обеспечиваем, чтобы скрытое состояние было непрерывным
        output, hidden = self.rnn(decoder_input_embedded, hidden)
        
        # 计算注意力权重    Вычисление весов внимания
        attention_weights = self.attn(output, encoder_outputs, mask)  # [batch_size, 1, seq_len]
        
        # 计算上下文向量    Вычисление контекстного вектора
        context = encoder_outputs.transpose(1, 2).bmm(attention_weights.permute(2, 0, 1))
        
        # 将上下文向量 squeeze 以匹配 output    Сжимаем контекстный вектор для соответствия output
        context = context.squeeze(2)  # [batch_size, hidden_dim]
        output = output.squeeze(1)  # [batch_size, hidden_dim]
        
        # 连接 output 和 context 并生成预测    Объединяем output и context и генерируем предсказания
        concat_input = torch.cat((output, context), 1)
        prediction = self.fc_out(concat_input)

        return prediction, hidden  # 返回预测和隐藏状态    Возвращаем предсказания и скрытое состояние


# 定义序列到序列模型类    Определение класса Seq2Seq модели
class Seq2SeqModelWithAttention(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2SeqModelWithAttention, self).__init__()
        self.encoder = encoder  # 编码器    Кодировщик
        self.decoder = decoder  # 解码器    Декодировщик

    def forward(self, encoder_input, decoder_input, teacher_forcing_ratio):
        encoder_outputs, hidden = self.encoder(encoder_input)  # 编码输入序列    Кодируем входную последовательность
    
        # 先检查 padding_idx 是否是一个 tensor 类型    Сначала проверяем, является ли padding_idx тензором
        padding_idx = torch.tensor(self.encoder.embedding.padding_idx, dtype=torch.long, device=encoder_input.device)
    
        # 获取 padding_idx 并创建掩码    Получаем padding_idx и создаем маску
        mask = (encoder_input != padding_idx)  # 比较是否为 padding_idx    Проверяем, является ли токен padding_idx
        mask = mask.float()  # 转换为浮动类型的张量    Преобразуем в тензор с плавающей точкой
        mask = mask.to(encoder_input.device)  # 确保掩码在相同的设备上    Обеспечиваем, чтобы маска была на том же устройстве
        mask = mask.permute(1, 0)  # 转置为 (seq_len, batch_size) 以匹配注意力层的要求    Переводим маску в (seq_len, batch_size), чтобы соответствовать требованиям слоя внимания
    
        outputs = torch.zeros(decoder_input.size(0), decoder_input.size(1), self.decoder.fc_out.out_features).to(decoder_input.device)  # 初始化输出序列    Инициализируем выходную последовательность
    
        input_token = decoder_input[:, 0].unsqueeze(1)  # 设置解码器的第一个输入标记为 <BOS>    Устанавливаем первый токен для декодера как <BOS>
    
        for t in range(1, decoder_input.size(1)):
            output, hidden = self.decoder(input_token, hidden, encoder_outputs, mask)  # 解码器输出    Выход декодера
            outputs[:, t, :] = output  # 存储当前时间步的输出    Сохраняем выход на текущем шаге времени
    
            # 使用教师强制决定下一个输入    Используем принудительное обучение для выбора следующего входа
            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            top1 = output.argmax(1).unsqueeze(1)  # 获取预测标记    Получаем предсказанный токен
            input_token = decoder_input[:, t].unsqueeze(1) if teacher_force else top1  # 使用教师强制或预测作为下一个输入    Используем принудительное обучение или предсказание как следующий вход
    
        return outputs  # 返回解码器的所有输出    Возвращаем все выходы декодера

In [27]:
# 设置超参数    Установка гиперпараметров
units = 256  # 隐藏单元的数量    Количество скрытых единиц
n_tokens = len(tokenizer.get_vocab())  # 输入词典大小    Размер словаря для входных данных
n_labels = len(tokenizer.get_vocab())  # 输出词典大小    Размер словаря для выходных данных
n_stacks = 3  # 编码器的层数    Количество слоев в кодировщике

# 初始化编码器，传入词典大小、隐藏单元数量、层数等参数    
# Инициализация кодировщика с параметрами размерности словаря, скрытых единиц и слоев
EncoderWithAttention = EncoderWithAttention(
    units=units,  # 隐藏单元的数量    Количество скрытых единиц
    n_tokens=n_tokens,  # 输入词典大小    Размер словаря для входных данных
    n_stacks=n_stacks,  # 编码器层数    Количество слоев в кодировщике
    bidirectional=True,  # 是否双向    Используется ли двусторонний RNN
    cell_type=nn.GRU,  # RNN的类型    Тип RNN (GRU)
    padding_idx=50258  # 填充标记的索引    Индекс токена для дополнения
)

# 初始化解码器，传入隐藏单元数量、输出词典大小、层数等参数    
# Инициализация декодировщика с параметрами скрытых единиц, размерности словаря и слоев
# 设定 attention_dim 和 embedding_dim 与 units 一致    Устанавливаем attention_dim и embedding_dim равными units
DecoderWithAttention = DecoderWithAttention(
    units=units,  # 隐藏单元的数量    Количество скрытых единиц
    n_labels=n_labels,  # 输出词典大小    Размер словаря для выходных данных
    attention_dim=units,  # 注意力层的维度    Размерность слоя внимания
    embedding_dim=units,  # 嵌入层的维度    Размерность слоя встраивания
    cell_type=nn.GRU  # RNN 的类型    Тип RNN (GRU)
)

# 初始化 seq2seq_model_WithAttention，将编码器和解码器实例传入    
# Инициализация модели seq2seq_model_WithAttention, передаем экземпляры кодировщика и декодировщика
seq2seq_model_WithAttention = Seq2SeqModelWithAttention(
    encoder=EncoderWithAttention,  # 编码器    Кодировщик
    decoder=DecoderWithAttention  # 解码器    Декодировщик
)

# 打印模型结构    Вывод структуры модели
print(seq2seq_model_WithAttention)

Seq2SeqModelWithAttention(
  (encoder): EncoderWithAttention(
    (embedding): Embedding(50260, 256, padding_idx=50258)
    (rnn): GRU(256, 256, num_layers=3, batch_first=True, dropout=0.3, bidirectional=True)
  )
  (decoder): DecoderWithAttention(
    (embedding): Embedding(50260, 256)
    (rnn): GRU(256, 256, batch_first=True)
    (fc_out): Linear(in_features=512, out_features=50260, bias=True)
    (attn): Attention()
    (context_linear): Linear(in_features=256, out_features=256, bias=True)
  )
)


In [28]:
# 定义模型的训练函数    Определение функции обучения модели
def train_model_with_attention(data_loader, model, criterion, optimizer, num_epochs, teacher_forcing_ratio, decay_factor=0.8):
    model.train()  # 设置模型为训练模式    Устанавливаем модель в режим обучения

    # 开始迭代训练    Начало цикла обучения
    for epoch in range(num_epochs):
        total_loss = 0  # 初始化总损失    Инициализация общей потери
        total_correct = 0  # 初始化总正确预测数    Инициализация общего количества правильных предсказаний
        total_tokens = 0  # 初始化总标记数    Инициализация общего числа токенов

        # 每个 epoch 后将教师强制比率逐渐降低    Понижение коэффициента принудительного обучения после каждого эпоха
        current_teacher_forcing_ratio = teacher_forcing_ratio * (decay_factor ** epoch)

        # 遍历数据加载器中的每个批次    Перебор каждого батча в загрузчике данных
        for batch in tqdm(data_loader, desc=f"Epoch [{epoch + 1}/{num_epochs}]"):
            input_ids = batch['input_ids'].to(device)  # 输入序列（来源于数据加载器）    Входные последовательности (из загрузчика данных)
            output_ids = batch['output_ids'].to(device)  # 输出序列（真实目标序列）    Выходные последовательности (истинные метки)
            attention_mask = batch['attention_mask'].to(device)  # 注意力掩码（用于忽略 padding 部分）    Массив маски внимания (для игнорирования паддинга)

            optimizer.zero_grad()  # 清除上一轮的梯度    Обнуляем градиенты с предыдущего шага

            # 通过模型进行前向传播，生成预测    Прогоняем данные через модель для получения предсказаний
            predictions = model(input_ids, output_ids, current_teacher_forcing_ratio)

            # 计算损失函数：将预测的 logits 展平，然后计算与真实标签的交叉熵损失    Вычисление потерь: преобразуем предсказанные логи и вычисляем кросс-энтропию
            loss = criterion(predictions.view(-1, predictions.size(-1)), output_ids.view(-1))
            loss.backward()  # 反向传播计算梯度    Обратное распространение для вычисления градиентов

            # 梯度裁剪 Обрезка градиентов
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)  # 防止梯度爆炸    Чтобы избежать взрыва градиентов

            optimizer.step()  # 更新模型参数    Обновление параметров модели

            # 累积损失    Накопление потерь
            total_loss += loss.item()

            # 计算准确率    Вычисление точности
            predicted_tokens = predictions.argmax(-1)  # 获取每个时间步的预测标记    Получаем предсказания для каждого временного шага
            correct = ((predicted_tokens == output_ids) * attention_mask).float().sum()  # 计算正确预测的标记数    Подсчитываем количество правильных предсказанных токенов
            total_correct += correct.item()  # 累加正确的标记数    Накопление правильных предсказаний
            total_tokens += attention_mask.sum().item()  # 累加有效标记的总数（忽略 padding）    Накопление общего числа токенов (игнорируя паддинг)

        # 每个 epoch 结束后清理缓存    Очистка кэша после каждого эпоха
        torch.cuda.empty_cache()

        # 计算并输出每个 epoch 的平均损失和准确率    Вычисление и вывод средней потери и точности для каждого эпоха
        epoch_loss = total_loss / len(data_loader)  # 计算平均损失    Вычисление средней потери
        epoch_accuracy = total_correct / total_tokens  # 计算准确率    Вычисление точности
        print(f"Epoch [{epoch + 1}/{num_epochs}]\n Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")

In [35]:
# 损失函数和优化器    Функция потерь и оптимизатор
criterion = nn.CrossEntropyLoss(ignore_index=50258)  # 定义交叉熵损失函数，忽略 padding 的 token    Определение функции потерь кросс-энтропии, игнорируя токены паддинга
optimizer2 = optim.Adam(seq2seq_model_WithAttention.parameters(), lr=learning_rate)  # 使用 Adam 优化器    Использование оптимизатора Adam

# 将模型转入设备    Перемещаем модель на устройство (GPU или CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # 判断是否有可用的 GPU，如果有则使用 GPU，否则使用 CPU    Проверяем, доступен ли GPU, если да, используем его, иначе используем CPU
seq2seq_model_WithAttention.to(device)  # 将模型转移到指定的设备    Перемещаем модель на выбранное устройство

Seq2SeqModelWithAttention(
  (encoder): EncoderWithAttention(
    (embedding): Embedding(50260, 256, padding_idx=50258)
    (rnn): GRU(256, 256, num_layers=3, batch_first=True, dropout=0.3, bidirectional=True)
  )
  (decoder): DecoderWithAttention(
    (embedding): Embedding(50260, 256)
    (rnn): GRU(256, 256, batch_first=True)
    (fc_out): Linear(in_features=512, out_features=50260, bias=True)
    (attn): Attention()
    (context_linear): Linear(in_features=256, out_features=256, bias=True)
  )
)

In [17]:
# 损失函数和优化器    Функция потерь и оптимизатор
criterion = nn.CrossEntropyLoss(ignore_index=50258)  # 定义交叉熵损失函数，忽略 padding 的 token    Определение функции потерь кросс-энтропии, игнорируя токены паддинга
optimizer2 = optim.Adam(seq2seq_model_WithAttention.parameters(), lr=learning_rate)  # 使用 Adam 优化器    Использование оптимизатора Adam

# 将模型转入设备    Перемещаем модель на устройство (GPU или CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # 判断是否有可用的 GPU，如果有则使用 GPU，否则使用 CPU    Проверяем, доступен ли GPU, если да, используем его, иначе используем CPU
seq2seq_model_WithAttention.to(device)  # 将模型转移到指定的设备    Перемещаем модель на выбранное устройство

# 每次运行训练代码前调用此函数    Вызываем эту функцию перед запуском обучения
clear_cache()  # 清理缓存    Очистка кэша

# 训练模型    Обучаем модель
train_model_with_attention(data_loader, seq2seq_model_WithAttention, criterion, optimizer2, num_epochs, teacher_forcing_ratio, decay_factor=0.8)

Epoch [1/2]: 100%|███████████████████████████████████████████████████████████████| 2874/2874 [7:48:15<00:00,  9.78s/it]


Epoch [1/2]
 Loss: 5.2387, Accuracy: 0.2427


Epoch [2/2]: 100%|███████████████████████████████████████████████████████████████| 2874/2874 [6:41:01<00:00,  8.37s/it]


Epoch [2/2]
 Loss: 4.4261, Accuracy: 0.3493


# Continue Trainning

In [24]:
# 保存 seq2seq_model 和优化器的状态
torch.save({
    'model_state_dict': seq2seq_model.state_dict(),
    'optimizer_state_dict': optimizer1.state_dict(),
}, 'seq2seq_model.pth')

print("Model and optimizer saved.")

Model and optimizer saved.


In [22]:
# 加载模型和优化器的状态
checkpoint = torch.load('seq2seq_model.pth')
seq2seq_model.load_state_dict(checkpoint['model_state_dict'])
optimizer1.load_state_dict(checkpoint['optimizer_state_dict'])

# 将模型移动到设备
seq2seq_model.to(device)

# 继续训练
num_epochs_plus = 3
train_model(data_loader, seq2seq_model, criterion, optimizer1, num_epochs_plus, teacher_forcing_ratio, decay_factor=0.8)

print("Model loaded and training resumed.")

Epoch [1/3]: 100%|███████████████████████████████████████████████████████████████████████| 2874/2874 [7:29:04<00:00,  9.38s/it]


Epoch [1/3]
 Loss: 3.3028, Accuracy: 0.4971


Epoch [2/3]: 100%|███████████████████████████████████████████████████████████████████████| 2874/2874 [6:05:30<00:00,  7.63s/it]


Epoch [2/3]
 Loss: 3.1887, Accuracy: 0.5162


Epoch [3/3]: 100%|███████████████████████████████████████████████████████████████████████| 2874/2874 [7:36:58<00:00,  9.54s/it]


Epoch [3/3]
 Loss: 3.1551, Accuracy: 0.5198
Model loaded and training resumed.


In [23]:
# 保存 seq2seq_model_WithAttention 和优化器的状态
torch.save({
    'model_state_dict': seq2seq_model_WithAttention.state_dict(),
    'optimizer_state_dict': optimizer2.state_dict(),
}, 'seq2seq_model_with_attention.pth')

print("Seq2SeqWithAttention model and optimizer saved.")

Seq2SeqWithAttention model and optimizer saved.


In [25]:
# 加载 seq2seq_model_WithAttention 和优化器的状态
checkpoint_attention = torch.load('seq2seq_model_with_attention.pth')
seq2seq_model_WithAttention.load_state_dict(checkpoint_attention['model_state_dict'])
optimizer2.load_state_dict(checkpoint_attention['optimizer_state_dict'])

# 将模型移动到设备
seq2seq_model_WithAttention.to(device)

# 继续训练
train_model(data_loader, seq2seq_model_WithAttention, criterion, optimizer2, num_epochs_plus, teacher_forcing_ratio, decay_factor=0.8)

print("Seq2SeqWithAttention model loaded and training resumed.")

Epoch [1/3]: 100%|███████████████████████████████████████████████████████████████████████| 2874/2874 [8:25:04<00:00, 10.54s/it]


Epoch [1/3]
 Loss: 3.6719, Accuracy: 0.4420


Epoch [2/3]: 100%|███████████████████████████████████████████████████████████████████████| 2874/2874 [8:34:29<00:00, 10.74s/it]


Epoch [2/3]
 Loss: 3.4385, Accuracy: 0.4763


Epoch [3/3]: 100%|███████████████████████████████████████████████████████████████████████| 2874/2874 [7:47:11<00:00,  9.75s/it]


Epoch [3/3]
 Loss: 3.3380, Accuracy: 0.4895
Seq2SeqWithAttention model loaded and training resumed.


In [28]:
# 保存 seq2seq_model 和 optimizer1 的状态
torch.save({
    'model_state_dict': seq2seq_model.state_dict(),
    'optimizer_state_dict': optimizer1.state_dict(),
}, 'seq2seq_model2.pth')

print("Seq2Seq model and optimizer saved after two trainings.")

Seq2Seq model and optimizer saved after two trainings.


In [29]:
# 保存 seq2seq_model_WithAttention 和 optimizer2 的状态
torch.save({
    'model_state_dict': seq2seq_model_WithAttention.state_dict(),
    'optimizer_state_dict': optimizer2.state_dict(),
}, 'seq2seq_model_with_attention2.pth')

print("Seq2SeqWithAttention model and optimizer saved after two trainings.")

Seq2SeqWithAttention model and optimizer saved after two trainings.


In [37]:
# 加载 Seq2SeqModel
checkpoint = torch.load('seq2seq_model2.pth')  # 加载保存的检查点
seq2seq_model = Seq2SeqModel(encoder=encoder, decoder=decoder)  # 将实例化的 encoder 和 decoder 传入模型
# 加载模型权重
seq2seq_model.load_state_dict(checkpoint['model_state_dict'])  # 加载保存的模型权重
seq2seq_model.eval()  # 切换到评估模式
# 加载优化器状态
optimizer1.load_state_dict(checkpoint['optimizer_state_dict'])  # 恢复优化器的状态


# 加载 Seq2SeqAttentionModel
checkpoint_attention = torch.load('seq2seq_model_with_attention2.pth')  # 加载保存的检查点
seq2seq_attention_model = Seq2SeqModelWithAttention(encoder=EncoderWithAttention, decoder=DecoderWithAttention)
# 加载模型权重
seq2seq_attention_model.load_state_dict(checkpoint_attention['model_state_dict'])  # 加载保存的模型权重
seq2seq_attention_model.eval()  # 切换到评估模式
# 加载优化器状态
optimizer2.load_state_dict(checkpoint_attention['optimizer_state_dict'])  # 恢复优化器的状态

# Testing

In [125]:
# 示例文本列表 - Пример списка текстов
sample_texts = [
    "Давайте что-нибудь попробуем!",  # 让我们试试吧！
    "Мне пора идти спать.",           # 我该睡觉了。
    "Что ты делаешь?",                # 你在做什么？
    "Что ты делаешь?",                # 你在做什么？
    "Собака бегает по траве",         # 狗在草地上奔跑
    "Небо и море - все синее."        # 天空和海洋全都是蓝色
]

In [155]:
def translate(
    text: str,  # 输入文本 - Входной текст
    tokenizer: GPT2Tokenizer,  # 自定义分词器 - Индивидуальный токенизатор
    model: torch.nn.Module,  # 已训练的模型 - Обученная модель
    max_len: int = 48,  # 最大输出长度 - Максимальная длина вывода
    confidence_threshold: float = 0.3,  # 置信度阈值 - Порог уверенности
    low_confidence_limit: int = 15  # 连续低置信度 token 数限制 - Ограничение для токенов с низкой уверенностью
) -> str:
    '''使用模型预测文本的翻译 - Предсказать перевод текста с помощью модели'''
    
    # 将输入文本转换为模型输入格式 - Преобразуем текст в формат, подходящий для модели
    input_ids = tokenizer.encode(text, return_tensors="pt").to(device)

    # 设置模型为评估模式 - Установка модели в режим оценки
    model.eval()

    with torch.no_grad():  # 关闭梯度计算，不影响模型权重 - Отключение расчета градиентов
        # 初始化输出 ID 序列，以 <BOS> 作为起始 token - Инициализация выходной последовательности ID, начиная с токена <BOS>
        output_ids = torch.tensor([[tokenizer.bos_token_id]], dtype=torch.long, device=device)

        # 连续低置信度 token 计数器 - Счетчик подряд идущих токенов с низкой уверенностью
        low_confidence_count = 0

        for _ in range(max_len):  # 限制生成最大长度为 max_len - Ограничиваем максимальную длину генерации
            if isinstance(model, Seq2SeqModel):  # 如果是 Seq2SeqModel 模型 - Если это модель Seq2SeqModel
                # 对于 Seq2SeqModel, 只需要 input_ids 和 output_ids - Для Seq2SeqModel нужно только input_ids и output_ids
                outputs = model(input_ids, output_ids)
            else:  # 对于带有注意力机制的 Seq2SeqModelWithAttention 模型 - Для Seq2SeqModelWithAttention
                # 对于带有注意力机制的模型，传入 teacher_forcing_ratio 参数 - Для модели с вниманием, передаем параметр teacher_forcing_ratio
                teacher_forcing_ratio = 0  # 可以根据需要修改这个值 - Можем изменить это значение по необходимости
                outputs = model(input_ids, output_ids, teacher_forcing_ratio=teacher_forcing_ratio)

            # 获取最后一个 token 的预测分布 - Получаем распределение вероятностей для последнего токена
            next_token_logits = outputs[:, -1, :]

            # 应用 softmax 获取分布概率 - Применяем softmax для получения распределения вероятностей
            next_token_probs = torch.softmax(next_token_logits, dim=-1)

            # 获取概率最高的 token 和对应的概率 - Получаем токен с наивысшей вероятностью и его вероятность
            next_token_prob, next_token_id = next_token_probs.max(dim=-1)

            # 检查置信度是否低于阈值 - Проверяем, ниже ли уверенность порогового значения
            if next_token_prob.item() < confidence_threshold:
                low_confidence_count += 1
                # 如果连续低置信度 token 数达到限制，则停止生成 - Останавливаем генерацию, если количество низкоуверенных токенов достигает предела
                if low_confidence_count >= low_confidence_limit:
                    break
            else:
                low_confidence_count = 0  # 重置低置信度计数器 - Сброс счетчика низкой уверенности

            # 将预测的 token 追加到输出序列 - Добавляем предсказанный токен в выходную последовательность
            output_ids = torch.cat([output_ids, next_token_id.unsqueeze(0)], dim=1)

            # 如果生成了 <EOS>，则停止生成 - Останавливаем генерацию, если был сгенерирован <EOS>
            if next_token_id.item() == tokenizer.eos_token_id:
                break

    # 使用分词器将生成的 ID 序列解码为文本 - Декодируем сгенерированную последовательность ID в текст с помощью токенизатора
    translation = tokenizer.decode(output_ids[0].tolist(), skip_special_tokens=True)

    # 后处理：去除多余的标点符号 - Обработка после перевода: удаляем лишние знаки препинания
    def clean_translation(translation):
        translation = re.sub(r'([!?.])\1+', r'\1', translation)  # 去掉重复的标点符号 - Убираем повторяющиеся знаки препинания
        return translation
    
    cleaned_translation = clean_translation(translation)  # 清理翻译结果 - Очищаем результат перевода

    return cleaned_translation  # 返回翻译 - Возвращаем перевод

In [156]:
# 对每个文本进行翻译并打印结果 - Переводим каждый текст и выводим результат
for text in sample_texts:
    translation = translate(text, tokenizer, seq2seq_model)  # 使用模型进行翻译 - Переводим текст с помощью модели
    print(f"Original: {text}")  # 原始文本 - Оригинальный текст
    print(f"Translation: {translation}")  # 翻译文本 - Перевод текста

Original: Давайте что-нибудь попробуем!
Translation: !Let's's try something.
Original: Мне пора идти спать.
Translation: !I time to to go bed early early.
Original: Что ты делаешь?
Translation: !What are you do doing?.".".".".".
Original: Что ты делаешь?
Translation: !What do you you? doing?."."."."."."
Original: Собака бегает по траве
Translation: !The dog dog is is running.
Original: Небо и море - все синее.
Translation: !The sky is, blue the red blue one.


In [172]:
def translateAtt(text, tokenizer, model, max_len=50, confidence_threshold=0.5, low_confidence_limit=5):
    input_ids = tokenizer.encode(text, return_tensors='pt').to(device)
    output_ids = torch.full((input_ids.size(0), 1), tokenizer.bos_token_id).to(device)

    translation = []
    teacher_forcing_ratio = 0.5

    for t in range(max_len):
        model.eval()

        if isinstance(model, Seq2SeqModelWithAttention):
            outputs = model(input_ids, output_ids, teacher_forcing_ratio=teacher_forcing_ratio)
            if isinstance(outputs, tuple):
                outputs, attention_weights = outputs
            else:
                attention_weights = None
            # 打印 attention_weights 和 outputs 来检查它们是否有变化
            print(f"Step {t}:")
            print(f"Attention Weights: {attention_weights}")
            print(f"Outputs: {outputs}")
        else:
            outputs = model(input_ids, output_ids)
            attention_weights = None

        predicted_token = outputs[:, -1, :].argmax(dim=-1)
        translation.append(predicted_token.item())

        if predicted_token.item() == tokenizer.eos_token_id:
            break

        output_ids = torch.cat((output_ids, predicted_token.unsqueeze(-1)), dim=1)

    translated_text = tokenizer.decode(translation, skip_special_tokens=True)
    return translated_text


In [173]:
# 对每个文本进行翻译并打印结果 - Переводим каждый текст и выводим результат
for text in sample_texts:
    translation = translateAtt(text, tokenizer, seq2seq_attention_model)  # 使用模型进行翻译 - Переводим текст с помощью модели
    print(f"Original: {text}")  # 原始文本 - Оригинальный текст
    print(f"Translation: {translation}")  # 翻译文本 - Перевод текста

Step 0:
Attention Weights: None
Outputs: tensor([[[0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:0')
Step 1:
Attention Weights: None
Outputs: tensor([[[   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
             0.0000],
         [ 416.0692, -118.8889, -346.5327,  ..., -243.5838, -250.9626,
          -244.5615]]], device='cuda:0', grad_fn=<CopySlices>)
Step 2:
Attention Weights: None
Outputs: tensor([[[   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
             0.0000],
         [ 416.0692, -118.8889, -346.5327,  ..., -243.5838, -250.9626,
          -244.5615],
         [ 418.5793, -118.4888, -349.5076,  ..., -249.2327, -256.4819,
          -250.3589]]], device='cuda:0', grad_fn=<CopySlices>)
Step 3:
Attention Weights: None
Outputs: tensor([[[   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
             0.0000],
         [ 416.0692, -118.8889, -346.5327,  ..., -243.5838, -250.9626,
          -244.5615],
         [ 417.6671, -118.8771, -349.0438, 